# Your Title Here

**Name(s)**: Tiantian Zheng, Ziling Xu

**Website Link**: (your website link)

In [62]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

# from dsc80_utils import * # Feel free to uncomment and use this.

## Step 1: Introduction

### Research Question: 
What characteristics are associated with more severe power outages in the United States? Specifically, how do factors such as geographic location, time of year, climate conditions, and other regional economic indicators influence the severity of power outages? Which of these factors could help energy companies predict the location and impact of future major outages?

## Step 2: Data Cleaning and Exploratory Data Analysis

In [63]:
!pip install openpyxl

In [64]:
# TODO
df = pd.read_excel('outage_dataset.xlsx',skiprows=5)
df

,variables,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
0,Units,NaN,NaN,NaN,NaN,NaN,NaN,NaN,numeric,NaN,...,%,%,persons per square mile,persons per square mile,persons per square mile,%,%,%,%,%
1,NaN,1.0,2011.0,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
2,NaN,2.0,2014.0,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
3,NaN,3.0,2010.0,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
4,NaN,4.0,2012.0,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,NaN,1530.0,2011.0,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765
1531,NaN,1531.0,2006.0,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765
1532,NaN,1532.0,2009.0,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.307744,1.692256,1.692256
1533,NaN,1533.0,2009.0,8.0,South Dakota,SD,MRO,West North Central,0.5,warm,...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.307744,1.692256,1.692256


In [65]:
df_no_unit = df.drop(index=0).drop(columns=["variables"])
df_no_unit

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
1,1.0,2011.0,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01 00:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
2,2.0,2014.0,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11 00:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
3,3.0,2010.0,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26 00:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
4,4.0,2012.0,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19 00:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
5,5.0,2015.0,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18 00:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,1530.0,2011.0,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,2011-12-06 00:00:00,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765
1531,1531.0,2006.0,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,NaN,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765
1532,1532.0,2009.0,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,2009-08-29 00:00:00,...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.307744,1.692256,1.692256
1533,1533.0,2009.0,8.0,South Dakota,SD,MRO,West North Central,0.5,warm,2009-08-29 00:00:00,...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.307744,1.692256,1.692256


In [66]:
df_no_unit.columns

Index(['OBS', 'YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE', 'NERC.REGION',
       'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY',
       'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE',
       'OUTAGE.RESTORATION.TIME', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL',
       'HURRICANE.NAMES', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW',
       'CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE',
       'TOTAL.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES', 'TOTAL.SALES',
       'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'RES.CUSTOMERS',
       'COM.CUSTOMERS', 'IND.CUSTOMERS', 'TOTAL.CUSTOMERS', 'RES.CUST.PCT',
       'COM.CUST.PCT', 'IND.CUST.PCT', 'PC.REALGSP.STATE', 'PC.REALGSP.USA',
       'PC.REALGSP.REL', 'PC.REALGSP.CHANGE', 'UTIL.REALGSP', 'TOTAL.REALGSP',
       'UTIL.CONTRI', 'PI.UTIL.OFUSA', 'POPULATION', 'POPPCT_URBAN',
       'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL',
       'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND', 'PCT_WATER_TOT',
    

In [90]:
df_cleaned = df_no_unit.get(['YEAR', 'MONTH','OUTAGE.START.TIME','U.S._STATE', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL','CUSTOMERS.AFFECTED', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW','POPULATION','TOTAL.PRICE', 'TOTAL.SALES','TOTAL.CUSTOMERS', 'PC.REALGSP.CHANGE'])
df_cleaned.isna().sum(axis=1)
df_cleaned.loc[:, "OUTAGE.DURATION"] = pd.to_numeric(df_cleaned["OUTAGE.DURATION"], errors='coerce')
df_cleaned = df_cleaned[df_cleaned['CUSTOMERS.AFFECTED'].notna()].reset_index(drop=True)

In [91]:
fig = px.histogram(df_no_unit, x='CAUSE.CATEGORY', nbins=30, title='Histogram of Cause of Power Outage')
fig.show()


In [92]:
fig2 = px.histogram(df_no_unit, x='CLIMATE.CATEGORY', nbins=30, title='Histogram of Climate of Power Outage')
fig2.show()

In [93]:
# Boxplot for OUTAGE.DURATION

fig2 = px.box(df, y='OUTAGE.DURATION', title='Outage Duration Distribution')
fig2.update_layout(yaxis_title='Duration (hours)')
fig2.show()

In [94]:
df_new = df_cleaned.copy()
df_new['OUTAGE.DURATION'] = df_new['OUTAGE.DURATION'].clip(upper=7020)

fig2 = px.box(df_new, y='OUTAGE.DURATION', title='Outage Duration Distribution (Cleaned)')
fig2.update_layout(yaxis_title='Duration (hours)')
fig2.show()

In [95]:
df_new = df_cleaned.copy()
numeric_cols = ["OUTAGE.DURATION"]

# Use IQR to identify outliers and remove these outliers
for col in numeric_cols:
    Q1 = df_new[col].quantile(0.25)
    Q3 = df_new[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
 
    df_new = df_new[(df_new[col] >= lower) & (df_new[col] <= upper)]
df_new

,YEAR,MONTH,OUTAGE.START.TIME,U.S._STATE,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,CUSTOMERS.AFFECTED,OUTAGE.DURATION,DEMAND.LOSS.MW,POPULATION,TOTAL.PRICE,TOTAL.SALES,TOTAL.CUSTOMERS,PC.REALGSP.CHANGE
0,2011.0,7.0,17:00:00,Minnesota,severe weather,NaN,70000.0,3060.0,NaN,5348119.0,9.28,6562520,2595696.0,1.6
1,2010.0,10.0,20:00:00,Minnesota,severe weather,heavy wind,70000.0,3000.0,NaN,5310903.0,8.15,5222116,2586905.0,2.7
2,2012.0,6.0,04:30:00,Minnesota,severe weather,thunderstorm,68200.0,2550.0,NaN,5380443.0,9.19,5787064,2606813.0,0.6
3,2015.0,7.0,02:00:00,Minnesota,severe weather,NaN,250000.0,1740.0,250,5489594.0,10.43,5970339,2673531.0,1.7
4,2010.0,11.0,15:00:00,Minnesota,severe weather,winter storm,60000.0,1860.0,NaN,5310903.0,8.28,5374150,2586905.0,2.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,2004.0,6.0,17:35:00,Idaho,system operability disruption,NaN,35000.0,95.0,157,1391802.0,5.13,2034123,701140.0,2.8
1085,2011.0,1.0,07:00:00,Idaho,intentional attack,vandalism,0.0,360.0,0,1584134.0,6.54,2063171,794925.0,-0.7
1086,2003.0,6.0,15:12:00,Idaho,public appeal,NaN,0.0,1548.0,0,1363380.0,4.94,2015509,687334.0,1.5
1087,2016.0,3.0,00:00:00,Idaho,intentional attack,sabotage,0.0,0.0,0,1680026.0,8.05,1657605,849763.0,2.7


In [96]:
# Full plot
fig3 = px.scatter(df_no_unit, x="OUTAGE.DURATION", y="CUSTOMERS.AFFECTED", title='Scatter Plot of Number of Customers Affected vs Outage Duration')
fig3.update_layout(
    xaxis=dict(range=[0, 50000]),
    height=800)
fig3.show()

In [74]:
fig4 = px.scatter(df_no_unit, x="OUTAGE.DURATION", y="DEMAND.LOSS.MW", title='Scatter Plot of Number of Customers Affected vs Outage Duration')
fig4.update_layout(
    xaxis=dict(range=[0, 10000]),
    yaxis=dict(range=[0, 1000000]),
    height=800)
fig4.show()

In [75]:
# Scatter Plot of Number of Customers Affected vs Outage Duration Zoomed in
fig4 = px.scatter(df_no_unit, x="OUTAGE.DURATION", y="CUSTOMERS.AFFECTED", title='Scatter Plot of Number of Customers Affected vs Outage Duration')
fig4.update_layout(
    xaxis=dict(range=[0, 10000]),
    yaxis=dict(range=[0, 1000000]),
    height=600)
fig4.show()

In [76]:
df_no_nan = df_new.dropna(subset=["CUSTOMERS.AFFECTED"])

# Now group and compute the mean
agg = df_no_nan.groupby(["CAUSE.CATEGORY", "CAUSE.CATEGORY.DETAIL"])["CUSTOMERS.AFFECTED"].mean()

agg2 = df_no_nan.groupby(["CAUSE.CATEGORY", "CAUSE.CATEGORY.DETAIL"]).size()

agg = agg.to_frame().reset_index()
agg2 = agg2.to_frame("Count").reset_index()


In [77]:
agg

,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,CUSTOMERS.AFFECTED
0,equipment failure,breaker trip,68500.000000
1,equipment failure,cables,120000.000000
2,equipment failure,generator trip,19321.000000
3,equipment failure,line fault,30000.000000
4,equipment failure,relaying malfunction,68530.000000
5,equipment failure,substation,182000.000000
6,equipment failure,switching,100000.000000
7,equipment failure,transformer outage,18400.000000
8,equipment failure,transmission,43903.000000
9,equipment failure,transmission interruption,51500.000000


In [78]:
agg2

,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,Count
0,equipment failure,breaker trip,2
1,equipment failure,cables,1
2,equipment failure,generator trip,6
3,equipment failure,line fault,1
4,equipment failure,relaying malfunction,1
5,equipment failure,substation,1
6,equipment failure,switching,1
7,equipment failure,transformer outage,1
8,equipment failure,transmission,1
9,equipment failure,transmission interruption,1


In [79]:
agg_final = pd.concat([agg, agg2["Count"]], axis=1)
agg_final

,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,CUSTOMERS.AFFECTED,Count
0,equipment failure,breaker trip,68500.000000,2
1,equipment failure,cables,120000.000000,1
2,equipment failure,generator trip,19321.000000,6
3,equipment failure,line fault,30000.000000,1
4,equipment failure,relaying malfunction,68530.000000,1
5,equipment failure,substation,182000.000000,1
6,equipment failure,switching,100000.000000,1
7,equipment failure,transformer outage,18400.000000,1
8,equipment failure,transmission,43903.000000,1
9,equipment failure,transmission interruption,51500.000000,1


In [80]:
agg_final["SEVERE_WEATHER"] = (agg_final["CAUSE.CATEGORY"] == "severe weather").astype(int)
agg_final


,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,CUSTOMERS.AFFECTED,Count,SEVERE_WEATHER
0,equipment failure,breaker trip,68500.000000,2,0
1,equipment failure,cables,120000.000000,1,0
2,equipment failure,generator trip,19321.000000,6,0
3,equipment failure,line fault,30000.000000,1,0
4,equipment failure,relaying malfunction,68530.000000,1,0
5,equipment failure,substation,182000.000000,1,0
6,equipment failure,switching,100000.000000,1,0
7,equipment failure,transformer outage,18400.000000,1,0
8,equipment failure,transmission,43903.000000,1,0
9,equipment failure,transmission interruption,51500.000000,1,0


## Step 3: Assessment of Missingness

In [97]:
df_cleaned.isnull().sum()

YEAR                       0
MONTH                      7
OUTAGE.START.TIME          7
U.S._STATE                 0
CAUSE.CATEGORY             0
CAUSE.CATEGORY.DETAIL    321
CUSTOMERS.AFFECTED         0
OUTAGE.DURATION           35
DEMAND.LOSS.MW           378
POPULATION                 0
TOTAL.PRICE               16
TOTAL.SALES               16
TOTAL.CUSTOMERS            0
PC.REALGSP.CHANGE          0
dtype: int64

## Step 4: Hypothesis Testing

In [82]:
# Step 1: Define Observed Test Statistic
natural = agg_final[agg_final["SEVERE_WEATHER"] == 1]["CUSTOMERS.AFFECTED"]
non_natural = agg_final[agg_final["SEVERE_WEATHER"] == 0]["CUSTOMERS.AFFECTED"]

observed_diff = natural.mean() - non_natural.mean()
print("Observed difference:", observed_diff)

Observed difference: 108544.95125862498


In [83]:
# Step 2: Permutation Test
diffs = []
n_reps = 5000

# Extract original customers affected as an array
affected = agg_final["CUSTOMERS.AFFECTED"].values
labels = agg_final["SEVERE_WEATHER"].values

for _ in range(n_reps):
    # Shuffle the labels
    shuffled_labels = np.random.permutation(labels)
    
    # Compute mean difference based on shuffled labels
    diff = affected[shuffled_labels == 1].mean() - affected[shuffled_labels == 0].mean()
    diffs.append(diff)

diffs = np.array(diffs)
diffs

array([31173.36594871, 51195.20262962, 55892.55190156, ...,
        2850.00942268, 14986.67471305, -5988.38477569])

In [84]:
# Step 3: p-value
p_value = np.mean(diffs >= observed_diff)
print("P-value:", p_value)


P-value: 0.0006


In [85]:
# Step 4: Visualize
import pandas as pd
import plotly.express as px

df_plot = pd.DataFrame({'Diff': diffs})
fig = px.histogram(df_plot, x="Diff", nbins=50, title="Null Distribution of Mean Differences")
fig.add_vline(x=observed_diff, line_dash="dash", line_color="red", annotation_text="Observed")
fig.show()

## Step 5: Framing a Prediction Problem

In [86]:
# TODO

## Step 6: Baseline Model

In [87]:
# TODO

## Step 7: Final Model

In [88]:
# TODO

## Step 8: Fairness Analysis

In [89]:
# TODO